In [12]:
import mediapipe as mp
import cv2
import time
import math
import numpy as np

mpHands = mp.solutions.hands
mpDraw = mp.solutions.drawing_utils

label = []

#From MEDIAPIPE DOCS

class HandDetector:
    def __init__(self, max_num_hands=3 , min_detection_confidence=0.7, min_tracking_confidence=0.8):

        self.tipIds = [4, 8, 12, 16, 20]
        self.mpDraw = mp.solutions.drawing_utils
        self.mpHands = mp.solutions.hands
        self.maxHands = max_num_hands
        self.hands = mpHands.Hands(max_num_hands=max_num_hands, min_detection_confidence=min_detection_confidence,
                                   min_tracking_confidence=min_tracking_confidence)
    
    #From MEDIAPIPE DOCS
    def findHandLandMarks(self, image, handNumber=0, draw=False):
        originalImage = image
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # mediapipe needs RGB
        results = self.hands.process(image)
        landMarkList = []

        if results.multi_handedness:
            label = results.multi_handedness[handNumber].classification[0].label  # label gives if hand is left or right
            # account for inversion in webcams
            if label == "Left":
                label = "Right"
            elif label == "Right":
                label = "Left"

        if results.multi_hand_landmarks:
            hand = results.multi_hand_landmarks[handNumber]  # results.multi_hand_landmarks returns landMarks for all the hands

            for id, landMark in enumerate(hand.landmark):
                # landMark holds x,y,z ratios of single landmark
                imgH, imgW, imgC = originalImage.shape  # height, width, channel for image
                xPos, yPos = int(landMark.x * imgW), int(landMark.y * imgH)
                landMarkList.append([id, xPos, yPos, label])

            if draw:
                mpDraw.draw_landmarks(originalImage, hand, mpHands.HAND_CONNECTIONS)

        return landMarkList
    
    #Taken from Murtaza Robotics Lab for moving the cursor
    def findHands(self, img, draw=True):
        imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        self.results = self.hands.process(imgRGB)
        # print(results.multi_hand_landmarks)

        if self.results.multi_hand_landmarks:
            for handLms in self.results.multi_hand_landmarks:
                if draw:
                    self.mpDraw.draw_landmarks(img, handLms,
                                               self.mpHands.HAND_CONNECTIONS)

        return img
    
    #Taken from Murtaza Robotics Lab for moving the cursor
    def findDistance(self, p1, p2, img, draw=True,r=15, t=3):
        x1, y1 = self.lmList[p1][1:]
        x2, y2 = self.lmList[p2][1:]
        cx, cy = (x1 + x2) // 2, (y1 + y2) // 2

        if draw:
            cv2.line(img, (x1, y1), (x2, y2), (255, 0, 255), t)
            cv2.circle(img, (x1, y1), r, (255, 0, 255), cv2.FILLED)
            cv2.circle(img, (x2, y2), r, (255, 0, 255), cv2.FILLED)
            cv2.circle(img, (cx, cy), r, (0, 0, 255), cv2.FILLED)
        length = math.hypot(x2 - x1, y2 - y1)

        return length, img, [x1, y1, x2, y2, cx, cy]
    
    #Taken from Murtaza Robotics Lab for moving the cursor
    def findPosition(self, img, handNo=0, draw=True):
        xList = []
        yList = []
        bbox = []
        self.lmList = []
        if self.results.multi_hand_landmarks:
            myHand = self.results.multi_hand_landmarks[handNo]
            for id, lm in enumerate(myHand.landmark):
                # print(id, lm)
                h, w, c = img.shape
                cx, cy = int(lm.x * w), int(lm.y * h)
                xList.append(cx)
                yList.append(cy)
                # print(id, cx, cy)
                self.lmList.append([id, cx, cy])
                if draw:
                    cv2.circle(img, (cx, cy), 5, (255, 0, 255), cv2.FILLED)

            xmin, xmax = min(xList), max(xList)
            ymin, ymax = min(yList), max(yList)
            bbox = xmin, ymin, xmax, ymax

            if draw:
                cv2.rectangle(img, (xmin - 20, ymin - 20), (xmax + 20, ymax + 20),
                              (0, 255, 0), 2)

        return self.lmList, bbox
    
    #Taken from Murtaza Robotics Lab for moving the cursor
    def fingersUp(self):
        fingers = []
        # Thumb
        if self.lmList[self.tipIds[0]][1] > self.lmList[self.tipIds[0] - 1][1]:
            fingers.append(1)
        else:
            fingers.append(0)

        # Fingers
        for id in range(1, 5):

            if self.lmList[self.tipIds[id]][2] < self.lmList[self.tipIds[id] - 2][2]:
                fingers.append(1)
            else:
                fingers.append(0)

        return fingers

import cv2
import pyautogui
import time
from pynput.mouse import Button, Controller
import autopy
import numpy as np
from mss import mss
from PIL import Image
import screen_brightness_control as sbc

handDetector = HandDetector(min_detection_confidence=0.7)
webcamFeed = cv2.VideoCapture(0)

mouse = Controller()

pyautogui.FAILSAFE = False

# Declaration of Right Hand Landmarks
right_thumb = 0
right_index = 0
right_middle = 0
right_ring = 0
right_little = 0

# Declaration of Right Hand Landmarks

left_thumb = 0
left_index = 0
left_middle = 0
left_ring = 0
left_little = 0

wCam, hCam = 640, 480
frameR = 100  # Frame Reduction
smoothening = 5

pTime = 0
plocX, plocY = 0, 0
clocX, clocY = 0, 0

############################################################################


def showImage(imgName):
    img = Image.open(imgName)
    img.show()


working1 = False
working2 = False
working3 = False
working4 = False
working5 = False
working6 = False
working7 = False
working8 = False

print("Below are the functionality you can select from")
time.sleep(1)
print("1. Switch Window")
time.sleep(0.5)
print("2. Space")
time.sleep(0.5)
print("3. Enter")
time.sleep(0.5)
print("4. Screenshot")
time.sleep(0.5)
print("5. Volume Up")
time.sleep(0.5)
print("6. Volume Down")
time.sleep(0.5)
print("7. Backspace")
time.sleep(0.5)
print("8. Brightness Up")
time.sleep(0.5)
print("9. Scroll Up")
time.sleep(0.5)
print("10. Scroll Down")
time.sleep(1.0)


#############################################################################


##################################################


def backspace():
    cv2.putText(image, "Backspace", (45, 375), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 5)  # BACKSPACE
    pyautogui.hotkey('backspace')
    time.sleep(0.2)


def volumeUp():
    cv2.putText(image, "Volume Increase", (45, 375), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 5)  # VOLUME UP
    pyautogui.press('volumeup')


def volumeDown():
    cv2.putText(image, "Volume Down", (45, 375), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 5)  # VOLUME DOWN
    pyautogui.press('volumedown')


def windowSwitch():
    cv2.putText(image, "WINDOW SWITCH", (45, 375), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 5)  # SWITCH WINDOW
    pyautogui.hotkey('ctrl', 'alt', 'tab')
    time.sleep(1)


def screenshot():
    cv2.putText(image, "MOUSE LEFT CLICK", (45, 375), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 5)  # SCREENSHOT
    filename = mss().shot()
    print(filename)
    time.sleep(1)


def enter():
    cv2.putText(image, "ENTER", (45, 375), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 5)  # ENTER
    pyautogui.press('enter')
    time.sleep(1)


def space() :
    cv2.putText(image, "SPACE", (45, 375), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 5)  # SPACE
    pyautogui.press('space')
    time.sleep(1)


def brightnessUp():
    cv2.putText(image, "BRIGHTNESS UP", (45, 375), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 5)  # BRIGHTNESS UP
    current_brightness = sbc.get_brightness()
    print(current_brightness)
    new_brightness = sbc.set_brightness(current_brightness + 5)
    print(new_brightness)
    
def brightnessDown():
    cv2.putText(image, "BRIGHTNESS Down", (45, 375), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 5)  # BRIGHTNESS Down
    curr_brightness = sbc.get_brightness()
    print(curr_brightness)
    baru_brightness = sbc.set_brightness(curr_brightness - 5)
    print(baru_brightness)
    
def scrollUp():
    cv2.putText(image, "Scroll Up", (45, 375), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 5)  # scroll UP
    pyautogui.scroll(50)

def scrollDown():
    cv2.putText(image, "Scroll Down", (45, 375), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 5)  # scroll UP
    pyautogui.scroll(-150)
    
def tutup():
    cv2.putText(image, "Close", (45, 375), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 5)  # scroll UP
    pyautogui.typewrite('q')
    
functions_dict = {
    '1': windowSwitch,
    '2': space,
    '3': enter,
    '4': screenshot,
    '5': volumeUp,
    '6': volumeDown,
    '7': backspace,
    '8': brightnessUp,
    '9': scrollUp,
    '10': scrollDown,
    '11': brightnessDown,
    '12': tutup
}
#################################################

while True:
    status, image = webcamFeed.read()
    # webcamFeed.set(3, wCam)
    # webcamFeed.set(4, hCam)
    wScr, hScr = autopy.screen.size()
    handLandmarks = handDetector.findHandLandMarks(image=image, draw=True)
    count = 0

    if len(handLandmarks) != 0:

        right_thumb = True if (handLandmarks[4][3] == "Right" and handLandmarks[4][1] > handLandmarks[3][1]) else False
        right_index = True if (handLandmarks[4][3] == "Right" and handLandmarks[8][2] < handLandmarks[6][2]) else False
        right_middle = True if (
                handLandmarks[4][3] == "Right" and handLandmarks[12][2] < handLandmarks[10][2]) else False
        right_ring = True if (handLandmarks[4][3] == "Right" and handLandmarks[16][2] < handLandmarks[14][2]) else False
        right_little = True if (
                handLandmarks[4][3] == "Right" and handLandmarks[20][2] < handLandmarks[18][2]) else False

        left_thumb = True if (handLandmarks[4][3] == "Left" and handLandmarks[8][2] < handLandmarks[6][2]) else False
        left_index = True if (handLandmarks[4][3] == "Left" and handLandmarks[8][2] < handLandmarks[6][2]) else False
        left_middle = True if (handLandmarks[4][3] == "Left" and handLandmarks[12][2] < handLandmarks[10][2]) else False
        left_ring = True if (handLandmarks[4][3] == "Left" and handLandmarks[16][2] < handLandmarks[14][2]) else False
        left_little = True if (handLandmarks[4][3] == "Left" and handLandmarks[20][2] < handLandmarks[18][2]) else False

        if right_thumb and right_little and not right_middle and not right_ring and not right_index:
            # volDown
            functions_dict['6']()
            
        elif not left_middle and not left_index and not left_little and not left_thumb and not left_ring:
            functions_dict['12']()

        elif right_thumb and not right_index and not right_middle and not right_ring and not right_little:
            functions_dict['8']()
            
        elif right_little and not right_index and not right_middle and not right_ring and not right_thumb:
            functions_dict['11']()

        elif left_thumb and left_index and not left_little and not left_ring and not left_middle:
            functions_dict['2']()

        elif right_thumb and right_index and right_little and right_middle and right_ring:
            functions_dict['7']()

        elif left_index and left_middle and not left_ring and not left_little and left_thumb:
            functions_dict['1']()

        elif right_index and right_middle and right_ring and not right_little and not right_thumb:
            functions_dict['4']()

        elif right_thumb and right_index and right_little and not right_middle and not right_ring:
            functions_dict['5']()

        elif left_thumb and left_index and left_middle and left_ring and left_little:
            functions_dict['3']()
            
        elif left_thumb and left_index and  left_little and not left_middle and not left_ring:
            functions_dict['10']()
            
            
        elif left_little and not left_thumb and not left_index and not left_middle and not left_ring:
            functions_dict['9']()
            
        

    image = handDetector.findHands(image)
    lmList, bbox = handDetector.findPosition(image)

    if len(lmList) != 0:
        x1, y1 = lmList[8][1:]
        x2, y2 = lmList[12][1:]
    cv2.rectangle(image, (frameR, frameR), (wCam - frameR, hCam - frameR), (255, 0, 255), 2)

    if right_index and not right_middle and not right_ring and not right_little and not right_thumb:
        cv2.putText(image, "MOVE CURSOR", (45, 375), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 5)  # MOVE CURSOR
        x3 = np.interp(x1, (frameR, wCam - frameR), (0, wScr))
        y3 = np.interp(y1, (frameR, hCam - frameR), (0, hScr))
        clocX = plocX + (x3 - plocX) / smoothening
        clocY = plocY + (y3 - plocY) / smoothening

        autopy.mouse.move(wScr - clocX, clocY)
        cv2.circle(image, (x1, y1), 15, (255, 0, 255), cv2.FILLED)
        plocX, plocY = clocX, clocY

    if right_index and right_middle and not right_ring and not right_little and not right_thumb:
        cv2.putText(image, "Click", (45, 375), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 5)  # CLICK
        length, img, lineInfo = handDetector.findDistance(10, 14, image + 5000)
        print(length)
        if length < 50:
            cv2.circle(img, (lineInfo[4], lineInfo[5]),
                       15, (0, 255, 0), cv2.FILLED)
            autopy.mouse.click()
   
    cv2.imshow("Webcam", image)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

webcamFeed.release()
cv2.destroyAllWindows()

Below are the functionality you can select from
1. Switch Window
2. Space
3. Enter
4. Screenshot
5. Volume Up
6. Volume Down
7. Backspace
8. Brightness Up
9. Scroll Up
10. Scroll Down
